In [249]:
# Script imports
import cleanup
import utils
import pandas as pd

import utils


# test = False
# accounts = final_account.cleanup()
# card = final_card.cleanup(test)
# clients = final_client.cleanup()
# districts = final_districts.cleanup()
# loans = final_loans.cleanup(test)
# trans = final_trans.cleanup(test)
# disp = pd.read_csv('../../project/banking_data/disp.csv', sep=';')


test = True
# accounts = final_account.cleanup()
# card = final_card.cleanup(test)
# clients = final_client.cleanup()
# districts = final_districts.cleanup()
loans = final_loans.cleanup(test)
# trans = final_trans.cleanup(test)
# disp = pd.read_csv('../../project/banking_data/disp.csv', sep=';')




In [118]:
### Merge everything with sequel queries
from pandasql import sqldf

distrs = districts.rename(columns={"code ": "district_id"})
current = clients.merge(distrs, how='left', on="district_id")

## Merge Disp
## Mover para cleanup

# q1 = "SELECT * FROM disp WHERE Type='OWNER'"
# disp = sqldf(q1)
current = current.merge(disp, how='left', on="client_id")

current.head()


### Merge accounts
current = current.merge(accounts, how='left', on="account_id")
current = current.rename(columns={"date_x": "loan_date", "date_y" : "account_date"})

current.isnull().sum()

q1 = "SELECT *, MAX(date) FROM trans GROUP BY account_id; "
trans3 = sqldf(q1)
trans3
# ## Merge transactions
# trans2 = trans.rename(columns={"type": "trans_type" , "amount":"trans_ammount"})
current = current.merge(trans3, how='left', on="account_id")

current.isnull().sum()

df =  loans.merge(current, how='left', on="account_id")

df.isnull().sum()



loan_id                                              0
account_id                                           0
date                                                 0
amount_x                                             0
duration                                             0
payments                                             0
status                                               0
client_id                                            0
birth_number                                         0
gender                                               0
district_id_x                                        0
name                                                 0
region                                               0
no. of inhabitants                                   0
no. of municipalities with inhabitants < 499         0
no. of municipalities with inhabitants 500-1999      0
no. of municipalities with inhabitants 2000-9999     0
no. of municipalities with inhabitants >10000        0
no. of cit

In [119]:
# df = cleanup.cleanup()
# df = cleanup.normalize_category(df)

df[df['balance'].isna()]

# df.drop(index=df.index[114], 
#         axis=0, 
#         inplace=True)


,loan_id,account_id,date,amount_x,duration,payments,status,client_id,birth_number,gender,...,district_id_y,frequency,date_x,trans_id,date_y,type_y,operation,amount_y,balance,MAX(date)


In [120]:
df.columns
df = cleanup.normalize_category(df)

In [121]:
### Drop columns not needed

# df2 = loans.drop(columns=['date'])
# df2 = utils.normalization(df2,'status')

df2 = df.drop(columns=['date','account_id','district_id_x','date_x','disp_id','client_id','birth_number','trans_id','date_x'])
df2 = utils.normalization(df2,'status')

df2.columns


Index(['loan_id', 'amount_x', 'duration', 'payments', 'status', 'gender',
       'name ', 'region', 'no. of inhabitants',
       'no. of municipalities with inhabitants < 499 ',
       'no. of municipalities with inhabitants 500-1999',
       'no. of municipalities with inhabitants 2000-9999 ',
       'no. of municipalities with inhabitants >10000 ', 'no. of cities ',
       'ratio of urban inhabitants ', 'average salary ',
       'unemploymant rate '95 ', 'unemploymant rate '96 ',
       'unemploymant_growth', 'no. of enterpreneurs per 1000 inhabitants ',
       'no. of commited crimes '95 ', 'no. of commited crimes '96 ',
       'crime_growth', 'total_crime', 'type_x', 'district_id_y', 'frequency',
       'date_y', 'type_y', 'operation', 'amount_y', 'balance', 'MAX(date)'],
      dtype='object')

In [257]:
### Split the model


from sklearn.model_selection import train_test_split
# from imblearn.over_sampling import SMOTE


# X = df2.drop(columns=['status'])
# y = df2['status']

loans = loans.drop(columns=["date"])

print(loans)

X = loans.drop(columns=['status'])
y = loans['status']



# transform the dataset

# oversample = SMOTE()
# X, y = oversample.fit_resample(X, y)

X_train, X_test, y_train, y_test = train_test_split(X,y,train_size=0.8,test_size=0.2)

len(X_test)


     loan_id  account_id  amount  duration  payments  status
0       5895        4473   93960        60      1566     NaN
1       7122       10365  260640        36      7240     NaN
2       6173        5724  232560        48      4845     NaN
3       6142        5591  221880        60      3698     NaN
4       5358        2018   38520        12      3210     NaN
..       ...         ...     ...       ...       ...     ...
349     4989         105  352704        48      7348     NaN
350     5221        1284   52512        12      4376     NaN
351     6402        6922  139488        24      5812     NaN
352     5346        1928   55632        24      2318     NaN
353     6748        8645  240900        60      4015     NaN

[354 rows x 6 columns]


71

In [196]:
### Train the Model

from sklearn.ensemble import RandomForestClassifier

model = RandomForestClassifier(n_estimators=15)
model.fit(X_train,y_train)



RandomForestClassifier(n_estimators=15)

In [197]:
model.score(X_test,y_test)

0.7878787878787878

In [258]:
y_predicted = model.predict(X_test)
y_predicted

array([-1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,
        1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,
        1,  1, -1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,
        1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1, -1,  1,  1,  1,  1,  1,
        1,  1,  1], dtype=int64)

In [259]:
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sn
from sklearn.metrics import confusion_matrix

cm = confusion_matrix(y_test,y_predicted)

plt.figure(figsize=(10,7))
# sn.heatmap(cm,annot=True)
plt.xlabel('Predicted')
plt.ylabel('True')

ValueError: Input contains NaN, infinity or a value too large for dtype('float64').

In [260]:
### Get AUC
from sklearn import metrics

fpr, tpr, _ = metrics.roc_curve(y_test, y_predicted)
roc_auc = metrics.auc(fpr, tpr)

roc_auc



# final = pd.DataFrame()
# final["loan_id"] =  X["loan_id"]
# final[""]






ValueError: Input contains NaN, infinity or a value too large for dtype('float64').

In [244]:
final_df = pd.DataFrame()

final_df["Id"] = loans["loan_id"]
final_df['Predicted'] = y


final_df =  utils.normalization(final_df,"Predicted")



final_df.to_csv("out.csv",index=False)
